In [2]:
def bin_power(chan, bands, Fs):
    
    """To compute the  power in each frequency bin specified by Band from FFT result of
    X. By default, X is a real signal.

    bands:
        a list
        boundary frequencies (in Hz) of bins. They can be unequal bins, e.g.
        [0.5,4,7,12,30] which are delta, theta, alpha and beta respectively.

    chan:
         a list
         Channel on which Spectral Entropy is calculated
    
    Fs
        an integer
        the sampling rate in physical frequency
        
    Returns
    -------
    Power -
        a list-
        spectral power in each frequency bin.
        
    Power_ratio - 
        a list-
        spectral power in each frequency bin normalized by total power in ALL
        frequency bins.
    """

    C = np.fft.fft(chan)
    C = abs(C)**2

    Power = np.zeros(len(bands) - 1)
    
    for Freq_Index in range(0, len(bands) - 1):
        
        Freq = float(bands[Freq_Index])
        Next_Freq = float(bands[Freq_Index + 1])
        Power[Freq_Index] = sum(C[int(np.floor(Freq / Fs * len(chan))): int(np.floor(Next_Freq / Fs * len(chan)))])
        
    Power_Ratio = Power / sum(Power)
    return Power, Power_Ratio

In [3]:
import numpy as np
import pickle as pickle
import pandas as pd
import math

from sklearn.preprocessing import normalize

import os
import time

In [4]:
bands = [4,8,13,16,30,45] 
window_size = 512 
step_size = 32  
sample_rate = 128 
User_nums = ['2','4','6','8','10','12']

In [5]:
def stft(user_no, bands, window_size, step_size, sample_rate):
    
    all_features=[]
    
    
    user_folder='../../../User'+ str(user_no) + '/'
    
    user_csv= os.listdir(user_folder)
    
    user_csv = user_csv[1:]
    
    user_labels_folder = user_folder + 'Label/'
    
    user_labels=os.listdir(user_labels_folder)
    
    all_data=[]
    all_labels=[]
    
    for csv in user_csv:
        all_data.append(csv)
    
    for lab in user_labels:
        all_labels.append(lab)
     
    
    
    for i in range(0,len(all_data)):
        
        data = pd.read_csv(user_folder + all_data[i])
        cols = data.columns
        cols = cols[:14]
        labels=pd.read_csv(user_labels_folder + all_labels[i])
        labels=labels.values[0][:4]
        
        
        start = 0;
        while start + window_size < data.shape[0]:
            
            temp_array = []
            temp_data = [] 
            
            for i in cols:
                
                X = data[i][start : start + window_size] 
                
                Y = bin_power(X, bands, sample_rate)
                
                temp_data = temp_data + list(Y[0])
                
            temp_array.append(np.array(temp_data))
            temp_array.append(labels)
            
            all_features.append(np.array(temp_array))    
            start = start + step_size
            
    all_features = np.array(all_features)
    
    np.save('../Dataset/STFTData/'+'EGCNNData' + str(user_no),all_features, allow_pickle=True, fix_imports=True)

In [ ]:
for i in User_nums:
    stft(i, bands, window_size, step_size, sample_rate)